In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import random

class ImageClassificationDataset(Dataset):
    def __init__(self, positive_dir, negative_dir, transform=None):
        self.positive_dir = positive_dir
        self.negative_dir = negative_dir
        self.transform = transform
        self.positive_files = [(os.path.join(positive_dir, f), 1) for f in os.listdir(positive_dir) if f.endswith('.png')]
        self.negative_files = [(os.path.join(negative_dir, f), 0) for f in os.listdir(negative_dir) if f.endswith('.png')]
        self.all_files = self.positive_files + self.negative_files

    def __len__(self):
        return len(self.all_files)

    def __getitem__(self, idx):
        file_path, label = self.all_files[idx]
        image = Image.open(file_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, label, os.path.basename(file_path)

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

positive_dir = "data/train_images/Train/Positive"
negative_dir = "data/train_images/Train/Negative"

# use an equal number of positive and negative samples for training
positive_files = [f for f in os.listdir(positive_dir) if f.endswith('.png')]
negative_files = [f for f in os.listdir(negative_dir) if f.endswith('.png')]
n = min(len(positive_files), len(negative_files))
positive_files = positive_files[:n]
negative_files = negative_files[:n]

# dataset = ImageClassificationDataset(positive_dir, negative_dir, transform)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
# suffle split train and test data 85/15
random.shuffle(positive_files)
random.shuffle(negative_files)
split = int(0.85 * n)
train_positive_files = positive_files[:split]
train_negative_files = negative_files[:split]
test_positive_files = positive_files[split:]

train_dataset = ImageClassificationDataset(positive_dir, negative_dir, transform)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = ImageClassificationDataset(positive_dir, negative_dir, transform)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True)

/home/johann/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 2)  # Output: 2 classes (positive and negative)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()

In [3]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels, _ in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_dataloader):.4f}")

print("Training complete.")

Epoch [1/15], Loss: 0.8146
Epoch [2/15], Loss: 0.6625
Epoch [3/15], Loss: 0.6116
Epoch [4/15], Loss: 0.5371
Epoch [5/15], Loss: 0.4575
Epoch [6/15], Loss: 0.4141
Epoch [7/15], Loss: 0.3859
Epoch [8/15], Loss: 0.3577
Epoch [9/15], Loss: 0.3279
Epoch [10/15], Loss: 0.3067
Epoch [11/15], Loss: 0.2990
Epoch [12/15], Loss: 0.2824
Epoch [13/15], Loss: 0.2632
Epoch [14/15], Loss: 0.2542
Epoch [15/15], Loss: 0.2539
Training complete.


In [4]:
#store the model in "data/train_images/Train/models/"
model_dir = "data/train_images/Train/models"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, "model.pth")
torch.save(model.state_dict(), model_path)
print(f"Model saved at {model_path}")

# Load the model
model = SimpleCNN()
model.load_state_dict(torch.load(model_path))
model.eval()

Model saved at data/train_images/Train/models/model.pth


SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [5]:
# Function to evaluate the model and output file names
def evaluate_model(model, test_dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    all_filenames = []

    with torch.no_grad():
        for images, labels, filenames in test_dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.numpy())
            all_labels.extend(labels.numpy())
            all_filenames.extend(filenames)

    return all_predictions, all_labels, all_filenames

# # Evaluate the model
# test_dataset = ImageClassificationDataset(positive_dir, negative_dir, transform)
# test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

predictions, labels, filenames = evaluate_model(model, test_dataloader)

positive_dir = "data/train_images/Train/Positive"
# Print the results
idx = 0
for filename, prediction, label in zip(filenames, predictions, labels):
    print(f"File: {filename}, Prediction: {prediction}, Label: {label}")
    # print the original image
    try:
        image = Image.open(os.path.join(positive_dir, filename))
        image.show()
    except:
        print("Image not found")
    #image = Image.open(os.path.join(positive_dir, filename))
    # image.show()
    idx += 1
    if idx == 20:
        break

# print accuracy
correct = sum([1 if p == l else 0 for p, l in zip(predictions, labels)])
accuracy = correct / len(labels)
print(f"Accuracy: {accuracy:.2f}")

File: mask_20241203-163804-498_117_146_30.png, Prediction: 1, Label: 1
File: mask_20241126-162109-143_x_-2_y_-23_rotation_322.png, Prediction: 1, Label: 0
Image not found
File: mask_20241126-144530-138_x_-13_y_-57_rotation_58.png, Prediction: 0, Label: 1
File: mask_20241204-092157-059_x_-45_y_-58_rotation_190.png, Prediction: 0, Label: 0
Image not found
File: mask_20241203-084310-762_x_-53_y_-7_rotation_89.png, Prediction: 0, Label: 0
Image not found
File: mask_20241202-164658-012_x_-36_y_-41_rotation_254.png, Prediction: 1, Label: 0
Image not found
File: mask_20241204-092650-340_x_-19_y_-23_rotation_197.png, Prediction: 0, Label: 0
Image not found
File: mask_20241202-114443-932_103_117_0.png, Prediction: 1, Label: 1
File: mask_20241202-145830-150_x_-33_y_-73_rotation_130.png, Prediction: 1, Label: 1
File: mask_20241203-164400-558_x_-23_y_-85_rotation_344.png, Prediction: 1, Label: 1
File: mask_20241202-104956-446_x_-30_y_-16_rotation_95.png, Prediction: 0, Label: 0
Image not found
Fil

Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.
Opening in existing browser session.


[180670:180670:0106/181650.339532:ERROR:object_proxy.cc(576)] Failed to call method: org.freedesktop.ScreenSaver.GetActive: object_path= /org/freedesktop/ScreenSaver: org.freedesktop.DBus.Error.NotSupported: This method is not part of the idle inhibition specification: https://specifications.freedesktop.org/idle-inhibit-spec/latest/
Created TensorFlow Lite XNNPACK delegate for CPU.
Attempting to use a delegate that only supports static-sized tensors with a graph that has dynamic-sized tensors (tensor#141 is a dynamic-sized tensor).
